In [4]:
import re
import collections
def Process(lines):
    sign_regrex = re.compile('[!"#$%&\'()*+,-./:;<=>?@[\\]^_`|＄＃＠£â€™é\n]')
    word_list = []
    text_list = []
    true_label = []
    for text in lines:
        true_label.append(text.split("\t")[0])
        text = text.split("\t")[1]
        text = sign_regrex.sub("", text)
        text = re.sub("(\d+)", r" \1 ", text)
        words = text.split(" ")
        words = list(filter(lambda x:x, words))#空リスト削除
        words = list(map(lambda x:x.lower(), words))#小文字にする
        word_list.extend(words)
        text_list.append(words)
    return word_list, text_list, true_label

def MakeDict(name):
    f = open("/Users/wenda/chapter06/{}.txt".format(name), "r")
    lines = f.readlines()
    f.close()
    word_list, _, _ = Process(lines)
    c = collections.Counter(word_list).most_common()
    word_dic = {}
    for id, word in enumerate(c, 1):
        if int(word[1]) < 2:
            word_dic[word[0]] = 0
        else:
            word_dic[word[0]] = id
    return word_dic

def Word2Code(name, word_dic):
    f = open("/Users/wenda/chapter06/{}.txt".format(name), "r")
    lines = f.readlines()
    lines.pop(0)#カラムの行を除く
    _, text_list, true_label = Process(lines)
    true_label = list(map(EncoderNN, true_label))
    result_list = []
    for text in text_list:
        code_list = []
        for word in text:
            try:
                code = word_dic[word]
            except:
                code = 0
            code_list.append(code)
        result_list.append(code_list)
    f = open("{}_code.txt".format(name), "w")
    i = 0
    for t1, t2, t3 in zip(true_label, text_list, result_list):
        if i==0:
            f.write(str(t1)+"\t"+" ".join(t2)+"\t"+" ".join(map(str, t3)))
            i = 1
        else:
            f.write("\n"+str(t1)+"\t"+" ".join(t2)+"\t"+" ".join(map(str, t3)))
    f.close()

def EncoderNN(sign):
    if sign == "b":
        code = 0
    elif sign == "t":
        code = 1
    elif sign == "e":
        code = 2
    elif sign == "m":
        code = 3
    return code

word_dic = MakeDict("train")
Word2Code("train", word_dic)
Word2Code("test", word_dic)
Word2Code("valid", word_dic)
